In [1]:
import os
import pandas as pd

In [2]:
output_folder = "/home/moemen/CMPUT624/labels"

# English

In [3]:
EN_folder_path = "/media/moemen/Stuff/project/data/ds003643-download/annotation/EN"

In [4]:
EN_tree_file = os.path.join(EN_folder_path, "lppEN_tree.csv")

In [5]:
EN_output_file = os.path.join(output_folder, "EN_labels.csv")

## Extract sentences from trees

In [6]:
tree_df = pd.read_csv(EN_tree_file, names=["tree"])
tree_df.head()

,tree
0,(S (SBAR (RB once) (WHADVP (WRB when)) (S (NP ...
1,(S (NP (PRP it)) (VP (VBD showed) (NP (DT a) (...
2,(S (NP (RB here)) (VP (VBZ is) (NP (NP (DT a) ...
3,(S (NP (PRP it)) (VP (VBD said) (PP (IN in) (N...
4,(S (S (IN so) (S (NP (PRP i)) (VP (VP (VBD tho...


In [9]:
tree_df.iloc[2]['tree']

'(S (NP (RB here)) (VP (VBZ is) (NP (NP (DT a) (NN copy)) (PP (IN of) (NP (DT the) (NN drawing))))))'

In [119]:
tree_df.shape

(1499, 1)

In [120]:
tree_df.iloc[0]["tree"]

'(S (SBAR (RB once) (WHADVP (WRB when)) (S (NP (PRP i)) (VP (VBD was) (ADJP (NP (CD six) (NNS years)) (JJ old))))) (NP (PRP i)) (VP (VBD saw) (SBAR (S (NP (NP (DT a) (JJ magnificent) (NN picture)) (PP (IN in) (NP (NP (DT a) (NN book)) (PP (IN about) (NP (DT the) (JJ primeval) (NN forest)))))) (VP (VBD called) (NP (JJ real) (NN life) (NNS stories)))))))'

In [121]:
def extract_sentence_EN_FR(tree):
    new_sentence = ""
    for letter in tree:
        if letter.isalpha() and letter.islower():
            new_sentence += letter
        else:
            new_sentence += " "
    new_sentence = " ".join(new_sentence.split())
    return new_sentence

In [122]:
tree_df["sentence"] = tree_df["tree"].apply(extract_sentence_EN_FR)

In [123]:
tree_df["sentence"]

0       once when i was six years old i saw a magnific...
1       it showed a boa constrictor swallowing a wild ...
2                           here is a copy of the drawing
3       it said in the book boa constrictors swallow t...
4       so i thought a lot about the adventures of the...
                              ...                        
1494                  wait a while exactly under the star
1495    if a child then comes to you if he laughs if h...
1496                                       so think of me
1497                             save me from this sorrow
1498            write to me quickly to tell me he is back
Name: sentence, Length: 1499, dtype: object

## Sentiment Analysis

In [126]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def get_sentiment(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    out = []
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        element = {"label": l, "score": s}
        out.append(element)
    return out

In [128]:
for i in range(tree_df.shape[0]):
    print(i)
    sentence = tree_df.iloc[i]["sentence"]
    sentiment_list = get_sentiment(sentence)
    for element in sentiment_list:
        tree_df.at[i, element["label"]] = element["score"]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [130]:
tree_df.tail()

,tree,sentence,joy,surprise,sadness,love,anger,fear
1494,(S (VP (VB wait) (NP (DT a) (NN while)) (ADVP ...,wait a while exactly under the star,0.261604,0.010440,0.020345,0.004771,0.171781,0.531058
1495,(S (SBAR (SBAR (IN if) (S (NP (DT a) (NN child...,if a child then comes to you if he laughs if h...,0.010139,0.003142,0.001074,0.000457,0.017434,0.967753
1496,(S (RB so) (VP (VB think) (PP (IN of) (NP (PRP...,so think of me,0.568045,0.006068,0.075355,0.007001,0.285434,0.058097
1497,(S (VP (VB save) (NP (PRP me)) (PP (IN from) (...,save me from this sorrow,0.026502,0.000781,0.950723,0.009325,0.010087,0.002583
1498,(S (S (VP (VB write) (PP (TO to) (NP (NP (PRP ...,write to me quickly to tell me he is back,0.604804,0.002151,0.053822,0.004182,0.308284,0.026758


In [131]:
tree_df.to_csv(EN_output_file, index=False)

### Other Models (to be used if needed)

In [14]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "if a child then comes to you if he laughs make sure he is okay"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) neutral 0.7282
2) positive 0.1565
3) negative 0.1153


In [29]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["it looked like this"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9842793941497803}]

# French

In [134]:
FR_folder_path = "/media/moemen/Stuff/project/data/ds003643-download/annotation/FR"
FR_tree_file = os.path.join(FR_folder_path, "lppFR_tree.csv")
FR_output_file = os.path.join(output_folder, "FR_labels.csv")

In [135]:
from deep_translator import GoogleTranslator

FR_translator = GoogleTranslator(source='fr', target='en')

In [142]:
def FR_translate(sentence):
    return FR_translator.translate(sentence)

In [137]:
tree_df = pd.read_csv(FR_tree_file, names=["tree"])
tree_df.head()

,tree
0,(S (Ssub (CS lorsque) (VN (CLS j') (V avais)))...
1,(S (NP (PRO ça)) (VN (V représentait)) (NP (DE...
2,(S (VN (V voilà)) (NP (DET la) (NC copie) (PP ...
3,(S (VN (CLS on) (V disait)) (PP (P dans) (NP (...
4,(S (VN (CLS j') (V ai)) (ADV alors) (VPpart (A...


In [138]:
tree_df.shape

(1480, 1)

In [139]:
tree_df["sentence"] = tree_df["tree"].apply(extract_sentence_EN_FR)

In [140]:
tree_df["sentence"]

0       sub lorsque j avais six ans j ai vu une fois u...
1       ça représentait un serpent boa rel qui avalait...
2                             voilà la copie de le dessin
3       on disait dans le livre rel les serpents boas ...
4       j ai alors part beaucoup réfléchi sur les aven...
                              ...                        
1475    rien de l univers n est semblable sub si quelq...
1476                                     regardez le ciel
1477    demandez vous le mouton d oui ou non int a t i...
1478                     et vous verrez comme tout change
1479    et aucune grande personne ne comprendra jamais...
Name: sentence, Length: 1480, dtype: object

In [143]:
tree_df["sentence_translated"] = tree_df["sentence"].apply(FR_translate)

In [149]:
tree_df["sentence_translated"]

'it was said in the real book that boa snakes swallow their prey whole without chewing it then they can no longer move and they sleep during the six months of their digestion'

In [150]:
for i in range(tree_df.shape[0]):
    print(i)
    sentence = tree_df.iloc[i]["sentence_translated"]
    sentiment_list = get_sentiment(sentence)
    for element in sentiment_list:
        tree_df.at[i, element["label"]] = element["score"]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [151]:
tree_df.head()

,tree,sentence,sentence_translated,joy,surprise,love,sadness,fear,anger
0,(S (Ssub (CS lorsque) (VN (CLS j') (V avais)))...,sub lorsque j avais six ans j ai vu une fois u...,sub when I was six years old I once saw a magn...,0.997398,0.001202,0.000386,0.000367,0.000333,0.000315
1,(S (NP (PRO ça)) (VN (V représentait)) (NP (DE...,ça représentait un serpent boa rel qui avalait...,it represented a boa rel snake which swallowed...,0.018569,0.006046,0.001773,0.009641,0.547393,0.416579
2,(S (VN (V voilà)) (NP (DET la) (NC copie) (PP ...,voilà la copie de le dessin,here is the copy of the drawing,0.461235,0.034109,0.005487,0.038866,0.217407,0.242896
3,(S (VN (CLS on) (V disait)) (PP (P dans) (NP (...,on disait dans le livre rel les serpents boas ...,it was said in the real book that boa snakes s...,0.115438,0.003226,0.002534,0.119662,0.264965,0.494175
4,(S (VN (CLS j') (V ai)) (ADV alors) (VPpart (A...,j ai alors part beaucoup réfléchi sur les aven...,I then thought a lot about the adventures of t...,0.914161,0.006543,0.001605,0.005666,0.037303,0.034722


In [152]:
tree_df.to_csv(FR_output_file, index=False)

# Chinese

In [153]:
CN_folder_path = "/media/moemen/Stuff/project/data/ds003643-download/annotation/CN"
CN_tree_file = os.path.join(CN_folder_path, "lppCN_tree.csv")
CN_output_file = os.path.join(output_folder, "CN_labels.csv")

In [154]:
def is_chinese_character(char):
    # Get the Unicode code point of the character
    code_point = ord(char)
    
    # Check if the code point is within the Chinese character range
    return 0x4e00 <= code_point <= 0x9fff

In [155]:
def extract_sentence_CN(tree):
    new_sentence = ""
    for letter in tree:
        if is_chinese_character(letter):
            new_sentence += letter
        else:
            new_sentence += " "
    new_sentence = "".join(new_sentence.split())
    return new_sentence

In [156]:
CN_translator = GoogleTranslator(source='zh-CN', target='en')
def CN_translate(sentence):
    return CN_translator.translate(sentence)

In [157]:
tree_df = pd.read_csv(CN_tree_file, names=["tree"])
tree_df.head()

,tree
0,(S (PP (P 当) (NP (S (S (NP (PRP 我)) (VP (ADVP ...
1,(S (NP (S (S (VP (VA 画))) (DEC 的))) (VP (VC 是)...
2,(S (LCP (NP (NN 页头)) (LC 上)) (VP (ADVP (AD 就))...
3,(S (LCP (NP (DP (DT 这) (CLP (M 本))) (NP (NN 书)...
4,(S (NP (PRP 它们)) (VP (ADVP (AD 就)) (VP (PP (P ...


In [158]:
tree_df.shape

(1577, 1)

In [159]:
tree_df["sentence"] = tree_df["tree"].apply(extract_sentence_CN)

In [160]:
tree_df["sentence"]

0                当我还只有六岁的时候在一本描写原始森林的名叫真实的故事的书中看到了一幅精彩的插画
1                                        画的是一条蟒蛇正在吞食一只大野兽
2                                             页头上就是那幅画的摹本
3                     这本书中写道这些蟒蛇把它们的猎获物不加咀嚼地囫囵吞下尔后就不能再动弹了
4                                    它们就在长长的六个月的睡眠中消化这些食物
                              ...                        
1572                                       请你们就在那颗星星底下等一等
1573    如果这时有个小孩子向你走来如果他笑着他有金黄色的头发如果当你问他问题时他不回答你们一定会猜得...
1574                                             那就请你们帮个忙
1575                                             不要让我这么忧伤
1576                                         赶快写信告诉我他又回来了
Name: sentence, Length: 1577, dtype: object

In [161]:
tree_df["sentence_translated"] = tree_df["sentence"].apply(CN_translate)

In [162]:
tree_df["sentence_translated"]

0       When I was only six years old, I saw a wonderf...
1       The painting shows a python devouring a large ...
2        On the top of the page is a copy of the painting
3       The book states that these pythons swallow the...
4       They digest this food during their six-month l...
                              ...                        
1572                          Please wait under that star
1573    If a child comes up to you at this moment, if ...
1574                            Then please do me a favor
1575                                 Don't make me so sad
1576            Write quickly and tell me he's back again
Name: sentence_translated, Length: 1577, dtype: object

In [163]:
for i in range(tree_df.shape[0]):
    sentence = tree_df.iloc[i]["sentence_translated"]
    sentiment_list = get_sentiment(sentence)
    for element in sentiment_list:
        tree_df.at[i, element["label"]] = element["score"]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [164]:
tree_df.head()

,tree,sentence,sentence_translated,joy,surprise,love,sadness,fear,anger
0,(S (PP (P 当) (NP (S (S (NP (PRP 我)) (VP (ADVP ...,当我还只有六岁的时候在一本描写原始森林的名叫真实的故事的书中看到了一幅精彩的插画,"When I was only six years old, I saw a wonderf...",0.996210,0.002367,0.000424,0.000397,0.000342,0.000259
1,(S (NP (S (S (VP (VA 画))) (DEC 的))) (VP (VC 是)...,画的是一条蟒蛇正在吞食一只大野兽,The painting shows a python devouring a large ...,0.018704,0.007035,0.004800,0.009614,0.233572,0.726275
2,(S (LCP (NP (NN 页头)) (LC 上)) (VP (ADVP (AD 就))...,页头上就是那幅画的摹本,On the top of the page is a copy of the painting,0.322817,0.064246,0.010373,0.043228,0.385310,0.174026
3,(S (LCP (NP (DP (DT 这) (CLP (M 本))) (NP (NN 书)...,这本书中写道这些蟒蛇把它们的猎获物不加咀嚼地囫囵吞下尔后就不能再动弹了,The book states that these pythons swallow the...,0.036191,0.004260,0.001945,0.036444,0.771164,0.149997
4,(S (NP (PRP 它们)) (VP (ADVP (AD 就)) (VP (PP (P ...,它们就在长长的六个月的睡眠中消化这些食物,They digest this food during their six-month l...,0.354318,0.003827,0.006782,0.197121,0.194967,0.242984


In [165]:
tree_df.to_csv(CN_output_file, index=False)